<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/myGPTlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# myGPTlab

This app helps to process texts (longform content) with ChatGPT via API.

## Start:
**Run all cells - that's it.**
- The notebooks checks automatically, if an initial setup (with PIP etc.) is necessary. The setup status is saved in the file 'installation.done'
- You can force the setup by deleting this file or by going to Seciton "Setup and Configuration" and checking "inital_setup_mode". Afterwards uncheck "Setup and Configuration".

## Working with myGPTlab
Lorem ipsum...


# Setup and Configuration

In [2]:
# @title Settings

# @markdown Default model
default_model = 'GPT-3.5' # @param ["GPT-3.5", "GPT-4"]

# @markdown Start Gradio webapp.
start_gradio_webapp = False # @param {type:"boolean"}

# @markdown Initial Setup Mode for pip install, fetch credentials etc.
initial_setup_mode = False # @param {type:"boolean"}

# @markdown Debug Mode for extensive logging.
debug_mode = True # @param {type:"boolean"}

# @markdown iOS Mode to develop helper functions, no Gradio.
# @markdown Useful for development on iOS, eg. with Carnets App
ios_mode = False # @param {type:"boolean"}

In [3]:
folders = {
    'audio': 'audio',
    'audio-chunks': 'audio/chunks',
    'transcript':'audio-transcript',
    'transcript-chunks': 'audio-transcript/chunks',
    'text-input': 'text-input',
    'text-input-backup': 'text-input-backup'
}

In [4]:
from ipywidgets import widgets
from IPython.display import Javascript, display, clear_output
notify_output = widgets.Output()
display(notify_output)
@notify_output.capture()
def popup(text):
    clear_output()
    display(Javascript("alert('{}')".format(text)))
#popup('Hello World!')

Output()

In [5]:
import os
if initial_setup_mode != True:
  if os.path.exists('installation.done'):
      initial_setup_mode = False
      print('No initial setup - forced by existing file "installation.done"')
  else:
    initial_setup_mode = True
    print('Starting automatic setup - forced by missing file "installation.done".\n\nEnter API Keys as JSON (in next notebook cell).')
    popup('Starting automatic setup. Enter API Keys as JSON (in next notebook cell).')
else:
  print('Starting setup.\n\nEnter API Keys as JSON (in next notebook cell).')
  popup('Starting setup. Enter API Keys as JSON (in next notebook cell).')

Starting automatic setup - forced by missing file "installation.done".

Enter API Keys as JSON (in next notebook cell).


In [6]:
if initial_setup_mode == True:
  #popup('Enter API Keys as JSON:')
  !wget -q bit.ly/aknip-colab-setup
  %run aknip-colab-setup
else:
  print('No initial setup.')

Secrets (JSON string): ··········
Credentials stored in env 'CREDS'. Get values via: 
creds = json.loads(os.getenv('CREDS')) 
key = creds['OpenAI']['v1']['credential']


In [7]:
creds = json.loads(os.getenv('CREDS'))
# openAI_key = creds['OpenAI']['v2']['credential']
# print(openAI_key)

In [8]:
if ios_mode == False:
  print('Mac')
else:
  print('iOS')

Mac


In [9]:
if initial_setup_mode == True:
  !pip install openai==0.27.7 yt-dlp==2023.7.6 librosa==0.10.0.post2 pickle-mixin==1.0.2 langchain==0.0.225 PyPDF2==3.0.1 PyMuPDF==1.22.5 -q
else:
  print('No initial setup.')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00


In [10]:
if (initial_setup_mode == True) and (ios_mode == False) :
  !pip install gradio -q
else:
  print('No initial setup / iOS.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.2/294.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.1 MB/s eta 0:0

In [11]:
if (initial_setup_mode == True) and (ios_mode == False) :
  %load_ext gradio
else:
  print('No initial setup / iOS.')

In [12]:
if initial_setup_mode == True:
  f= open('installation.done','w+')
  f.close()
  print('Initial setup done. Application starting.')
  popup('Initial setup done. Application starting.')
else:
  print('No initial setup.')

Initial setup done. Application starting.


# Helper Functions

- **create_file_directory**: Creates a new directory - if it not exists yet. The always_delete flag forces a deletion even if it exists.

In [13]:
# v2 - 06.08.2023
import shutil
import os

def create_file_directory(directory, always_delete=False):
  # Creates a new directory - if it not exists yet. The always_delete flag forces a deletion even if it exists.
  # Examples:
  # - create_file_directory('texts', False) => creates a new directory only if it not exists yet
  # - create_file_directory('texts', True) => always deletes existing directory and creates a new one
  if os.path.exists(directory):
    if always_delete:
      # delete the diectory recursively
      shutil.rmtree(directory)
  # create directory
  if not os.path.exists(directory):
    os.mkdir(directory)


def find_files(path, extensions=[".txt"], recursive=False):
    # Recursively (optional) find all files with extension in path
    my_files = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if extensions == []:
                my_files.append(os.path.join(root, f))
            else:
                for ext in extensions:
                    if f.endswith(ext):
                        my_files.append(os.path.join(root, f))
        # no recursion / don't look inside any subdirectory
        if recursive == False:
            break
    return my_files


def merge_textfiles(path, extensions=[".txt"], recursive=False, new_filename='merged.txt'):
    # Recursively (optional) find all files with extension in path
    my_files = find_files(path, extensions, recursive)
    merged_text = ''
    for filename in my_files:
      # print(filename)
      f= open(filename,'r')
      if f.mode == 'r':
            contents =f.read()
      f.close()
      merged_text = merged_text + contents + '\n\n\n'

    f= open(new_filename,'w+')
    f.write(merged_text)
    f.close()

# The App

In [14]:
if (ios_mode == False) and (start_gradio_webapp == True):
  import gradio as gr

  # Theming
  theme = gr.themes.Default(
      primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
  )
  # Styling: Change max width
  css = """
    .gradio-container {max-width: 700px!important}
    .vspacer1 {margin-top: 50px}
  """

  with gr.Blocks(theme=theme, css=css) as demo:

      gr.Markdown("# ChatGPTLab 2.0", elem_classes="vspacer1")
      gr.Markdown("### Optimizing your work with LLMs.")

      project_name = gr.Textbox(label="Project name")

      #
      # 1. Input Text
      #
      with gr.Tab("Input Text "):
        gr.Markdown("Please enter text")

        # Input text via UI
        gr.Markdown("### Input your text:")
        text_input = gr.Textbox(label="Enter text", placeholder="Your text here...", lines=10)
        text_output = gr.Textbox(label="Result")

        def text_save(text, proj_name):
          create_file_directory(proj_name, False)
          create_file_directory(proj_name + '/' +  folders['text-input'], False)
          f= open(proj_name + '/' +  folders['text-input'] + '/input_text.txt','w+')
          f.write(text)
          f.close()
          log_text = "Text saved."
          return log_text
        text_button = gr.Button("Save text")
        text_button.click(text_save, [text_input, project_name], text_output)

        gr.Markdown("")
        gr.Markdown("")

        # Input text via upload
        gr.Markdown("### Or upload your text:")
        upload_button = gr.UploadButton("Click to Upload a File", file_types=[".txt",".md"], file_count="single")
        file_output = gr.Textbox(label="Result")

        def upload_file(my_file, proj_name):
          create_file_directory(proj_name, False)
          create_file_directory(proj_name + '/' +  folders['text-input'], False)
          # copy to project directory
          full_upload_path = my_file.name
          just_the_filename = os.path.basename(full_upload_path)
          full_text_path = "./" + proj_name + '/' + folders['text-input'] + '/' + just_the_filename
          shutil.copyfile(full_upload_path, full_text_path)
          # check if file is empty
          f= open(full_text_path,'r')
          if f.mode == 'r': contents =f.read()
          f.close()
          log_text = just_the_filename + "\n"
          if len(contents) == 0:
            log_text = log_text + "Error: Upload file lengt 0 bytes"
          else:
            log_text = log_text + "Upload successful"
          return log_text
        upload_button.upload(upload_file, [upload_button, project_name], file_output)

      #
      # 2. Download full project
      #
      with gr.Tab("Download"):
        gr.Markdown("Download full project as ZIP file.")
        download_button = gr.Button("Download project")
        download_output = gr.File()

        def download_do(proj_name):
          full_text_path = "./" + proj_name
          shutil.make_archive('archive', 'zip', full_text_path)
          result = "Downloading " + full_text_path
          return "archive.zip"
        download_button.click(download_do, project_name, download_output)

      #
      # 3. xxx
      #
      with gr.Tab("Step 2"):
        gr.Markdown("Please select the optimization:")
        radio = gr.Radio(
          ["by headline", "by paragraph", "by §§§"], label="Text split method"
        )
        name = gr.Textbox(label="Name", placeholder="Enter text...")
        output = gr.Textbox(label="Output Box")
        greet_btn = gr.Button("Start", scale=0)
        def greet(name):
          result = "HALLO " + name + "!!!"
          return result
        greet_btn.click(fn=greet, inputs=name, outputs=output, api_name="greet")

  demo.launch(quiet=True, share=True, debug=debug_mode)

else:
  print('iOS Mode - Nothing to do.')

iOS Mode - Nothing to do.


# ChatGPT

In [106]:
# Promptlib
# Todo: promptlib not as global,

import pickle

# myprompt = "The given text is delimited by triple backticks. Summarize the current text to succint and clear bullet points of its contents.
#          The length of the summary must be 200 words maximum."
# myprompt = "The given text is delimited by triple backticks.
#           Summarize the current text to a maximum of 15 succint and clear bullet points of its contents."
# myprompt = myprompt + "The maximum number of words should be 300 words in total. "
# myprompt = myprompt + "Write everything in German language. " # this is optional !
# myprompt = myprompt + "```" + input_text + "```"



promptlib = {
        'sum': {
            'description': 'Max word length: 1100 for EN, 900 for DE. approx. 1 min processing time Example: Full book: 113 parts á 1000 words takes 15 min.',
            'category': 'summarize',
            '1': {
                'note': 'summarize prompt v1',
                'prompt': 'The given text is delimited by triple backticks. Summarize the current text to succint and clear bullet points of its contents. {language_text} {length_text} Text:```{input_text}```',
                'lang-de': 'Write everything in German language.',
                'lang-en': 'write everything in EN.',
                'lang-same': 'write in language of original text.',
                'length-max-fix': 'The length of the summary must be 200 words maximum.',
                'length-max-dyn': 'max length is {{length_calc("{input_text}", {max_len})}} words.'
            },
            '2': {
                'note': 'summarize prompt v2',
                'prompt': 'The given text is delimited by triple backticks. Summarize the current text to a maximum of 15 succint and clear bullet points of its contents. {language_text} {length_text} Text:```{input_text}```',
                'lang-de': 'Write everything in German language.',
                'lang-en': 'write everything in EN.',
                'lang-same': 'write in language of original text.',
                'length-max-fix': 'The maximum number of words should be 300 words in total.',
                'length-max-dyn': 'max length is {{length_calc("{input_text}", {max_len})}} words.'
            }
        },
        'para': {
            'description': 'paraphrase prompt',
            'category': 'paraphrase',
            '1': {
                'note': 'summarize prompt',
                'prompt': 'hello, this is prompttext para v1. {language_text} {length_text} Text:```{input_text}```',
                'lang-de': 'write evertyhing in DE.',
                'lang-en': 'write everything in EN.',
                'lang-same': 'write in language of original text.',
                'length-max-fix': 'max length is 100 words.',
                'length-max-dyn': 'max length is {{length_calc("{input_text}", {max_len})}} words.'
            }
        }
    }

def write_prompt_to_lib(prompt):
    # adds or updates prompt to library
    id = prompt['id']
    version = prompt['version']
    if id not in promptlib:
        # new id
        promptlib[id] = {}
        promptlib[id][version] = {}
    else:
        # id already existing, checking for version
        if version not in promptlib[id]:
            promptlib[id][version] = {}
    promptlib[id]['description'] = prompt['description']
    promptlib[id]['category'] = prompt['category']
    promptlib[id][version]['note'] = prompt['note']
    promptlib[id][version]['prompt'] = prompt['prompt']


def get_prompt_from_lib(id=None, version=None):
    # searches for prompt with given id
    # looks for highest available version, if no version is given
    if id==None:
        prompt_found = None
    else:
        prompt = promptlib[id]
        if version != None:
            # version given
            version_highest = int(version)
        else:
            # look for highest available version
            version_highest = 0
            for key, val in prompt.items():
                try:
                    version = int(key)
                except ValueError:
                    version = 0
                if version > version_highest:
                    version_highest = version
        prompt_version = prompt[str(version_highest)]
        # return dict
        prompt_found = {}
        prompt_found['id'] = id
        prompt_found['description'] = prompt['description']
        prompt_found['category'] = prompt['category']
        prompt_found['version'] = str(version_highest)
        prompt_found['note'] = prompt_version['note']
        prompt_found['prompt'] = prompt_version['prompt']
        prompt_found['lang-de'] = prompt_version['lang-de']
        prompt_found['lang-en'] = prompt_version['lang-en']
        prompt_found['lang-same'] = prompt_version['lang-same']
        prompt_found['length-max-fix'] = prompt_version['length-max-fix']
        prompt_found['length-max-dyn'] = prompt_version['length-max-dyn']
    return prompt_found

def build_prompt_from_template(input_txt, prompt_obj, lang, len='', len_max=1):
  prompt_text =  prompt_obj['prompt']
  language_text = prompt_obj[lang]
  # calculate max. output lengt dynamically
  if len != '':
    length_template = prompt_obj[len]
    length_template2 = length_template.format(input_text = input_txt, max_len=len_max)
    length_text = eval(f"f'{length_template2}'")
  else:
    length_text = ''
  final_prompt = prompt_text.format(language_text=language_text, length_text=length_text, input_text=input_txt)
  return final_prompt

def length_calc(my_text, maxWords):
  # maxWords 0.0 - 1.0 (percentage)
    return int(len(my_text) * maxWords)

def save_promptlib(promptlib):
  # saves promptlib to disk
  with open('promptlib.dictionary', 'wb') as dict_file:
      pickle.dump(promptlib, dict_file)

def load_promptlib():
  # load promptlib from disk
  with open('promptlib.dictionary', 'rb') as dict_file:
      promptlib = pickle.load(dict_file)
  return promptlib

def wordcount(input_text):
    # returns number of words of given input_text
    return len(input_text.split())

def split_text_by_separator (text, separator, joiner, fixer, min_words=1000, max_words=1000):
    # Split text into an array of texts with a maximum word count
    # example 1: Split by paragraphs
    #   split_text_by_separator(input_text, '\', '\', 'fix-nothing', 2000)
    # example 2: Split by sentences '. ' and fix
    #   split_text_by_separator(input_text, '. ', ' ', 'fix-end', 2000)
    # example 3: Split by markdown headlines '\n#' and fix
    #   split_text_by_separator(input_text, '\n#', '\n\n', 'fix-start', 2000)
    paragraphs = []
    sections = []
    section = ''
    # Split text, separated by separator
    paragraphs_stripped = text.split(separator)
    # fix paragraphs (if wanted) by re-adding the separator at end or start
    if fixer == 'fix-nothing':
        paragraphs = paragraphs_stripped
    if fixer == 'fix-end':
        for index, paragraph in enumerate(paragraphs_stripped):
            if (index+1) == len(paragraphs_stripped):
                paragraphs.append(paragraph)
            else:
                paragraphs.append(paragraph + separator)
    if fixer == 'fix-start':
        for index, paragraph in enumerate(paragraphs_stripped):
            if index == 0:
                paragraphs.append(paragraph)
            else:
                paragraphs.append(separator + paragraph)
    # Loop through paragraphs and aggregate up to maximum word count
    for index, paragraph in enumerate(paragraphs):
        if min_words == max_words:
            max_random = min_words
        else:
            max_random = random.randrange(min_words, max_words)
        test_section = section + paragraph + joiner
        if wordcount(test_section) > max_random:
            sections.append(section.strip())
            section = paragraph
            if min_words == max_words:
                max_random = min_words
            else:
                max_random = random.randrange(min_words, max_words)
        else:
            section = section + paragraph + joiner
            # if last paragraph, append to array
            if (index+1) == len(paragraphs):
              sections.append(section)
    return sections

In [ ]:
my_text = 'Die Photosynthese ist ein physiologischer Prozess zur Erzeugung energiereicher Biomoleküle aus energieärmeren Stoffen mit Hilfe\
 von Lichtenergie. Sie wird von Pflanzen, Algen und manchen Bakterien betrieben. Bei diesem biochemischen Vorgang wird Lichtenergie mit Hilfe\
 von lichtabsorbierenden Farbstoffen wie Chlorophyll in chemische Energie umgewandelt. Diese wird dann genutzt, um aus energiearmen\
 anorganischen Stoffen (vor allem Kohlenstoffdioxid (CO2) und Wasser (H2O)) energiereiche organische Verbindungen (vor allem Kohlenhydrate)\
 aufzubauen. Der genutzte Anteil der eingestrahlten Energie, nämlich der zum Aufbau der Assimilate verwendete Anteil, wird photosynthetische\
 Effizienz genannt. Soweit die energiereichen organischen Stoffe zu Bestandteilen des Lebewesens werden, bezeichnet man deren Synthese als\
 Assimilation. Man unterscheidet zwischen oxygener und anoxygener Photosynthese. Bei der oxygenen Photosynthese wird molekularer\
 Sauerstoff (O2) freigesetzt. Bei der anoxygenen Photosynthese, die nur von Bakterien betrieben wird, entstehen statt Sauerstoff andere\
 anorganische Stoffe, beispielsweise elementarer Schwefel (S). Die Photosynthese ist der einzige biochemische Prozess, bei dem\
 Lichtenergie, meistens Sonnenlicht, in chemisch gebundene Energie umgewandelt wird (Phototrophie). Indirekt sind auch fast alle\
 heterotrophen (nicht zur Photosynthese fähigen) Lebewesen von ihr abhängig, da sie der Photosynthese letztlich ihre Nahrung und auch\
 den zur Energiegewinnung mittels aerober Atmung benötigten Sauerstoff verdanken. Aus dem Sauerstoff entsteht außerdem die schützende\
 Ozonschicht der Erdatmosphäre.'
#print(my_text)
print(textwrap.fill(my_text, 120) + '\n')

In [ ]:
my_text2 = f' '

In [88]:
# Create summary

from datetime import datetime
import langchain
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.schema import HumanMessage
from langchain.callbacks.base import BaseCallbackHandler

# OpenAI Key
openAI_key = creds['OpenAI']['v2']['credential']

def create_summary_bullets (input_text):
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openAI_key)
  #llm = ChatOpenAI(model_name="gpt-4", temperature=0.0, openai_api_key=openAI_key)
  # model overveiew see https://gptforwork.com/guides/openai-gpt3-models

  input_text_segments = input_text.split("§§§")
  output_text_segments = []
  for index, input_text in enumerate(input_text_segments):

      # model_name="gpt-3.5-turbo", temperature=0.0
      prompt_obj = get_prompt_from_lib('sum', '2')
      print(json.dumps(prompt_obj, sort_keys=False, indent=2) + '\n')
      #prompt_txt_final = build_prompt_from_template(input_text, prompt_obj, 'lang-de', 'length-max-dyn', 0.5)
      #prompt_txt_final = build_prompt_from_template(input_text, prompt_obj, 'lang-de', 'length-max-fix')
      prompt_txt_final = build_prompt_from_template(input_text, prompt_obj, 'lang-de')
      print(textwrap.fill(prompt_txt_final, 120) + '\n')

      print('Processing step ' + str(index + 1) + ' of ' + str(len(input_text_segments)))
      print('Started at: ' + datetime.now().strftime("%H:%M:%S"))
      gpt_response_obj = llm.generate([[HumanMessage(content=prompt_txt_final)]])
      print('Ended at: ' + datetime.now().strftime("%H:%M:%S"))
      tmp_text = gpt_response_obj.generations[0][0].text
      tmp_tokens = gpt_response_obj.llm_output
      #st.write(tmp_text)
      print(str(tmp_tokens) + '\n')
      output_text_segments.append(tmp_text)

  full_output_text = '\n\n'.join(output_text_segments)
  return full_output_text

result_text = create_summary_bullets(my_text)
print(result_text)


{
  "id": "sum",
  "description": "summarize prompt",
  "category": "summarize",
  "version": "2",
  "note": "summarize prompt v2",
  "prompt": "The given text is delimited by triple backticks. Summarize the current text to a maximum of 15 succint and clear bullet points of its contents. {language_text} {length_text} Text:```{input_text}```",
  "lang-de": "Write everything in German language.",
  "lang-en": "write everything in EN.",
  "lang-same": "write in language of original text.",
  "length-max-fix": "The maximum number of words should be 300 words in total.",
  "length-max-dyn": "max length is {{length_calc(\"{input_text}\", {max_len})}} words."
}

The given text is delimited by triple backticks. Summarize the current text to a maximum of 15 succint and clear bullet
points of its contents. Write everything in German language.  Text:```Die Photosynthese ist ein physiologischer Prozess
zur Erzeugung energiereicher Biomoleküle aus energieärmeren Stoffen mit Hilfe von Lichtenergie. 

In [83]:
# test promptlib and templates

import textwrap

prompt_obj = get_prompt_from_lib('sum', '2')
# prompt_obj = get_prompt_from_lib('sum', '1')
# print(json.dumps(prompt_obj, sort_keys=True, indent=2))

input_text = 'This is the input text. Lorem ipsum.'

prompt_txt_final = build_prompt_from_template(input_text, prompt_obj, 'lang-de', 'length-max-dyn', 0.5)
prompt_txt_final = build_prompt_from_template(input_text, prompt_obj, 'lang-de', 'length-max-fix')
prompt_txt_final = build_prompt_from_template(input_text, prompt_obj, 'lang-de')
print(textwrap.fill(prompt_txt_final, 120))

#write_prompt_to_lib(prompt_obj)

#print(json.dumps(promptlib, sort_keys=True, indent=2))

# myprompt = "The given text is delimited by triple backticks. Summarize the current text to succint and clear bullet points of its contents.
#          The length of the summary must be 200 words maximum."
# myprompt = "The given text is delimited by triple backticks.
#           Summarize the current text to a maximum of 15 succint and clear bullet points of its contents."
# myprompt = myprompt + "The maximum number of words should be 300 words in total. "
# myprompt = myprompt + "Write everything in German language. " # this is optional !
# myprompt = myprompt + "```" + input_text + "```"

The given text is delimited by triple backticks. Summarize the current text to a maximum of 15 succint and clear bullet
points of its contents. Write everything in German language.  Text:```This is the input text. Lorem ipsum.```
